In [1]:
import tqdm
import snscrape.modules.twitter as tweet
import pandas as pd
import datetime
import re

In [ ]:
import mysql.connector
conn = mysql.connector.connect(host='localhost',
                        database='mastersstudents',
                        user='root',
                        password='root')

cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("SELECT DATABASE()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
cursor = conn.cursor()

In [ ]:
# sheet_names = ["Fall","Spring","StudentJob", "StudentOnCampus","OnCampusEmployment","OnCampusEmploymentStats"]
sheey_names = ["OnCampusEmployment"]

for sheet in sheet_names:
    df = pd.read_excel("D:\Darshana\\NEU\Study\DMDD\Project\MStudents\Assignment4\Mstudents.xlsx", sheet_name= sheet)
    c = 0
    for i, row in df.iterrows():
        try:
            if sheet == "Student":
                query = """INSERT INTO Student (StudentID, FirstName, LastName, Gender, DeptId, FeePayment, CreditsEarned) VALUES({0}, "{1}", "{2}", "{3}", {4}, "{5}", {6});""".format(int(row['StudentID']), row['FirstName'], row['LastName'], row['Gender'], int(row['DeptID']), row['FeePayment'],int(row['CreditsEarned']))
            
            if sheet == "Fall":
                query = """INSERT INTO Fall (StudentID, IntakeYear,ID) VALUES({0}, {1}, {2})""".format(int(row['StudentID']),int(row['IntakeYear']),int(row['ID']))
                
            if sheet == "Spring":
                query = """INSERT INTO Spring (StudentID, IntakeYear,ID) VALUES({0}, {1}, {2})""".format(int(row['StudentID']),int(row['IntakeYear']),int(row['ID']))
                
            if sheet == "College":
                query = """INSERT INTO College (CollegeName, DeptName, DeptID, DeanName) VALUES("{0}", "{1}", {2}, "{3}")""".format(row['CollegeName'], row['DeptName'], int(row['DeptID']),row['DeanName'])
                
            if sheet == "EventDetails":
                query = """INSERT INTO EventDetails(EID, EName, EventHeadID, Location, Purpose, Dateandtime, FareinUSD) VALUES({0},"{1}",{2},"{3}","{4}","{5}","{6}")""".format(int(row['EID']), row['EName'], int(row['EventHeadID']), row['Location'],row['Purpose'],datetime.datetime.strptime(row['Dateandtime'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"), row['FareinUSD'])
            
            if sheet == "OnCampusEmployment":
                query = """INSERT INTO OnCampusEmployment(JobID, Employer, Location) VALUES({0},"{1}","{2}")""".format(int(row['JobID']), row['Employer'], row['Location'])
                
            if sheet == "OnCampusEmploymentStats":
                query = """INSERT INTO OnCampusEmploymentStats(ID, JobID, SkillsRequired, SalaryInUSD, DurationInMonths, PostingDate, Deadline, Vacancy) VALUES({0},{1},"{2}",{3},{4},"{5}","{6}",{7})""".format(int(row['ID']), int(row['JobID']), row['SkillsRequired'], int(row['SalaryInUSD']),int(row['DurationInMonths']), datetime.datetime.strptime(row['PostingDate'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"), datetime.datetime.strptime(row['Deadline'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"),int(row['Vacancy']))
                
            if sheet == "StudentOnCampus":
                query = """INSERT INTO StudentOnCampus(StudentID, JobID,ID) VALUES({0},{1},{2})""".format(int(row['StudentID']), int(row['JobID']),int(row['ID']))

            if sheet == "InternshipsAndCoop":
                salary = row["SalaryEstimate"].replace("(Glassdoor est.)", "").replace("K", "000").replace("$","")
                salary = salary.split("-")
#                 import ipdb; ipdb.set_trace()
                lower_salary = salary[0]
                upper_salary = salary[1]
                rating = row['Rating']
                company_name = row['CompanyName'].replace('\n','').replace(str(rating),'')
                job_description = row['JobDescription'][:900]
                job_description = re.sub(r"\"","\'", job_description)
                job_title = re.sub(r"\"","\'", row['JobTitle'])
                query = """INSERT INTO InternshipsAndCoop(JobID, JobTitle, SalaryEstimateLower, SalaryEstimateUpper, JobDescription, Rating, CompanyName, Location, HeadQuaters, Size, FoundedYear, TypeOfOwnership, Industry, Sector, Revenue, Competitor) VALUES({0}, "{1}",{2},{3},"{4}",{5},"{6}","{7}","{8}","{9}",{10},"{11}","{12}","{13}","{14}","{15}")""".format(int(row['JobID']), job_title, int(lower_salary), int(upper_salary), job_description,float(row['Rating']), company_name,row['Location'], row['HeadQuarters'],row['Size'],int(row['Founded']),row['TypeOfOwnership'],row['Industry'],row['Sector'], row['Revenue'],row['Competitors']) 
                
            if sheet == "StudentClub":
                query = """INSERT INTO StudentClub (ClubID, ClubName, StudentID, Contacts, Motive) VALUES({0},"{1}",{2},"{3}","{4}")""".format(int(row['ClubID']), row['ClubName'], row['StudentID'], row['Contacts'], row['Motive'])
                
            if sheet == "StudentJob":
                query = "INSERT INTO StudentJob (StudentID, JobID, ID) VALUES({0}, {1},{2})".format(int(row['StudentID']), int(row['JobID']),int(row['ID']))
            cursor.execute(query)

        except Exception as e:
            print (e, sheet)
#             print (query)
#             break
            c+=1
    print (c)
    conn.commit()